In [135]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [134]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score,confusion_matrix,recall_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split,cross_val_score, KFold
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer


In [136]:
train=pd.read_csv('../input/titanic/train.csv')
test=pd.read_csv('../input/titanic/test.csv')
train.shape,test.shape

In [137]:
print(train.info())
print('---'*5)
print('percentage of NA')
print('---'*5)
miss=train.isna().sum().sort_values(ascending=False)
percent=(train.isna().sum()/train.isna().count()).sort_values(ascending=False)
d=pd.concat([miss,percent],axis=1,keys=['miss','percentage'])
print(d)
print('---'*5)
print('unique values')
print('---'*5)
print(train.nunique().sort_values())


## EDA

pclass: A proxy for socio-economic status (SES) 1st = Upper 2nd = Middle 3rd = Lower

age: Age is fractional if less than 1. If the age is estimated, is it in the form of xx.5

sibsp: The dataset defines family relations in this way... Sibling = brother, sister, stepbrother, stepsister Spouse = husband, wife (mistresses and fiancés were ignored)

parch: The dataset defines family relations in this way... Parent = mother, father Child = daughter, son, stepdaughter, stepson Some children travelled only with a nanny, therefore parch=0 for them.

Embarked is Port of Embarkation.C = Cherbourg, Q = Queenstown, S = Southampton


In [ ]:
train.head(3)

In [138]:
train['Embarked'].value_counts()
#fillna is equal to SimpleImputer(strategy='most_frequent')

In [139]:
train.Survived.value_counts().plot(kind='bar')
    

In [140]:
import seaborn as sns
ax=pd.crosstab(train['Sex'],train['Survived']).plot(kind='bar',stacked=True,rot=0)

for i in ax.containers:
    
    # set the bar label
    ax.bar_label(i, label_type='center')

plt.show()


In [141]:
a=sns.heatmap(pd.crosstab(train.Sex,train.Survived,normalize='index'),
           cmap='YlGnBu',
           annot=True,
            vmin=0.01
           )
a.set_xticklabels(['no','yes'])

In [142]:
pd.crosstab(train.Embarked,train.Survived).plot(kind='bar',stacked=True)

In [143]:
sns.heatmap(pd.crosstab(train.Embarked,train.Survived,normalize='index'),
           cmap='YlGnBu',
            annot=True,
            vmin=.01
           
           )

In [144]:
pd.crosstab(train.Pclass,train.Survived).plot(kind='bar')

In [145]:
import warnings
warnings.filterwarnings("ignore")

In [146]:
#we groupby and look at the ratio
train.groupby(['Sex','Embarked','Survived']).count()['PassengerId']/train.groupby(['Sex','Embarked'])['PassengerId'].count()

It seems the survival rate of female is higher than male

We then take a look at numeric variables and category variables. In this case, we can filter those by hand. But when we encounter a large amount of features, it would be better to use syntax. 

In [147]:

# because passengerid and pclass are not strictly numeric values we concern
# we excluded them
numeric=['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numcol=[i for i in train.columns if train[i].dtype in numeric and i !='Survived'
       and i !='Pclass' and i !='PassengerId']
numcol

In [148]:

catcol=[i for i in train.columns if train[i].dtype not in numeric ]
catcol

In [149]:
r,c=4,2
fig,ax=plt.subplots(nrows=r,ncols=c,figsize=(12,7))

for i,j in enumerate(numcol):
    sns.kdeplot(train.loc[train.Survived==0,j],ax=ax[i,0],shade=True)
    sns.kdeplot(train.loc[train.Survived==1,j],ax=ax[i,0],shade=True)
    sns.histplot(data=train,x=train[j],hue='Survived',ax=ax[i,1],bins=12,multiple='stack',kde=False)


We will look at the correlations and draw a heatmap of the correlations, but it doesn't contain infomation of category features.

In [150]:
cor=train.corr().abs()

sns.heatmap(cor,annot=True,fmt='.2f',
            cmap='coolwarm',
            mask=np.triu(cor),
           cbar_kws={'shrink':.8},vmin=0,vmax=1)
plt.show()


## Feature Engineering

In [151]:
train.info()

In [152]:
test.info()

In [153]:
train.head(3)

In [154]:
d  # the missing columns we summary before

————————————————————————————

As we can see(also in EDA), most of our features are complete and no missing values. But some columns indeed have missing values.

In Cabin, 77% are missed, we could drop it.

In Age column, 20% are missed, we could find some ways to fill in them.

The Embarked column is with 0.22% missing,we can fill in them or simply drop those rows.

Pay attention to our categories variables:

Sex is a binary variable, we can encode it by LabelEncoder or get_dummies or One Hot Encoder.

We could drop Name, or we could generate new features using Name, like Mr, Ms, Master, etc. I'll cover this in later version.

Cabin is also unuseful, but as we mentioned above, with 70% valus missing, we could simply drop it.

Ticket is also of no use, we drop it.

Embarked can be encoded with encoding commands.

We could use pipeline to impute missing values, or we could use hands.

In [155]:
catcol
# we already used Name to generate Title
# and we drop name, ticket, cabin (missing too many)

In [156]:
def clean(data):
    data.drop(['Name','Cabin','Ticket'],axis=1,inplace=True)
    data['Age']=data.groupby(['Pclass','Sex'])['Age'].transform(lambda x: x.fillna(x.mean()))
    data['Embarked']=data.groupby(['Pclass','Sex'])['Embarked'].fillna('S')
    data['Fare']=data.groupby(['Pclass','Sex'])['Fare'].transform(lambda x: x.fillna(x.mean()))
    
    return data

In [157]:
import copy
a=b=copy.deepcopy(train)
c=d=copy.deepcopy(test)

In [158]:
cleantrain=clean(train)
cleantest=clean(test)

In [159]:
cleantrain.info()

In [160]:
y=cleantrain['Survived']
X=pd.get_dummies(cleantrain.drop('Survived',axis=1))
X.shape,y.shape

In [161]:
xtrain,xtest,ytrain,ytest=train_test_split(X,y,test_size=.2,random_state=4)


## Model Selection

In [162]:
def mypredict(model):
    #evaluate models
    model.fit(xtrain, ytrain)
    prediction=model.predict(xtest)
    return accuracy_score(ytest,prediction),recall_score(ytest,prediction,average='weighted')


model1=LogisticRegression(solver='liblinear',random_state=4)
model2=XGBClassifier()   #pay attention to the brackets here
model3=RandomForestClassifier()
model4=SGDClassifier()
model5=SVC()

models=[model1,model2,model3,model4,model5]

res={'model':[],'acc':[],'recall':[]}

for m in models:
    res['model'].append(m)
    res['acc'].append(mypredict(m)[0])
    res['recall'].append(mypredict(m)[1])
    
res=pd.DataFrame(res)
res

We can then use RandomForest to predict and work on our test set.

## Make Prediction

In [163]:
x=pd.get_dummies(cleantest)
x.info()

In [164]:

out=model3.predict(x)
output=pd.DataFrame({'Id':cleantest.PassengerId,'Survived':out})
output.to_csv('output0316.csv',index=False)